In [1]:
from __future__ import absolute_import, division, print_function

import argparse
import os
import random
from datetime import datetime

import cv2
import keras
import numpy as np
import pandas as pd
from keras import backend as K
from keras.applications import *
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import *
from keras.datasets import fashion_mnist
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import *
from keras.utils.vis_utils import model_to_dot
from scipy import misc
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


In [20]:
def load_data():
    num_classes = 10
    (train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    
    train = train.reshape((train.shape[0],28,28,1))
    x_test = x_test.reshape((x_test.shape[0],28,28,1))

    # devide into train and validation sets
    dvi = int(train.shape[0] * 0.9)
    x_train = train[:dvi, :, :, :]
    y_train = y_train[:dvi]
    x_val = train[dvi:, :, :, :]
    y_val = y_train[dvi:]
    
    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_val /= 255
    x_test /= 255
    
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_val.shape[0], 'validation samples')
    print(x_test.shape[0], 'test samples')

    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

In [21]:
def create_base_network(input_shape, MODEL):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = input
    cnn_model = MODEL(
        include_top=False, input_shape=input_shape, weights=None)
    x = cnn_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu', name='sim')(x)
    model = Model(input, outputs=x)
    model.load_weights(model_name + '.h5', by_name=True)
    return model

In [22]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [59]:
# Loading Datasets
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_data()
input_shape = x_train.shape[1:]

num_classes = 10
input_shape = x_train.shape[1:]

x_train shape: (54000, 28, 28, 1)
54000 train samples
6000 validation samples
10000 test samples


In [60]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)

In [65]:
# network definition
base_network = create_base_network(input_shape, MobileNet)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

In [66]:
# base_network.summary()

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)


In [ ]:
# train
epochs = 200
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])

patience = 5
model_name = 'MobileNet'
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        # self.val_losses = []
        self.losses = []

    def on_epoch_end(self, batch, logs={}):
        # self.val_losses.append(logs.get("val_loss"))
        self.losses.append((logs.get('loss'), logs.get("val_loss")))
history = LossHistory()
early_stopping = EarlyStopping(
    monitor='val_loss', patience=patience, verbose=2, mode='auto')
checkpointer = ModelCheckpoint(
    filepath=model_name + '_sim.h5', verbose=0, save_best_only=True)
reduce_lr = ReduceLROnPlateau(factor=0.3, patience=3)

model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y),
          callbacks=[history, early_stopping, checkpointer, reduce_lr])




Train on 107320 samples, validate on 19980 samples
Epoch 1/200
107320/107320 [==============================] - 60s 563us/step - loss: 0.3137 - accuracy: 0.7402 - val_loss: 0.1345 - val_accuracy: 0.8327
Epoch 2/200
107320/107320 [==============================] - 55s 514us/step - loss: 0.1176 - accuracy: 0.8470 - val_loss: 0.1279 - val_accuracy: 0.8320
Epoch 3/200
107320/107320 [==============================] - 55s 514us/step - loss: 0.1018 - accuracy: 0.8679 - val_loss: 0.0927 - val_accuracy: 0.8766
Epoch 4/200
107320/107320 [==============================] - 55s 514us/step - loss: 0.1081 - accuracy: 0.8622 - val_loss: 0.1058 - val_accuracy: 0.8670
Epoch 5/200
107320/107320 [==============================] - 55s 516us/step - loss: 0.0948 - accuracy: 0.8804 - val_loss: 0.0880 - val_accuracy: 0.8896
Epoch 6/200
107320/107320 [==============================] - 55s 516us/step - loss: 0.0830 - accuracy: 0.8970 - val_loss: 0.0855 - val_accuracy: 0.8917
Epoch 7/200
107320/107320 [==========

In [ ]:
# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))